# Univariate Search Method

In [114]:
!sage --version

SageMath version 10.5, Release Date: 2024-12-04


In [8]:
from sage.all import var, solve, N
from sage.all import *
import operator
import random

In [9]:
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [16]:
x, y, z, a, b = var('x y z a b')

In [119]:
func = lambda a, b: 6*a**2 - 6*a*b + 2*b**2 - a - 2*b

In [120]:
def axis_search(func, point, axis, step=1e-2): # search along e1 and e2

    t = var('t')
    coords = list(point)
    base_val = float(func(*coords))

    # +step along axis
    forward = float(func(*(coords[:axis] + [coords[axis] + step] + coords[axis+1:])))
    # -step along axis
    backward = float(func(*(coords[:axis] + [coords[axis] - step] + coords[axis+1:])))

    if forward < backward and forward < base_val:
        search = func(*(coords[:axis] + [coords[axis] + t] + coords[axis+1:]))
        direction = 1
    elif backward < forward and backward < base_val:
        search = func(*(coords[:axis] + [coords[axis] - t] + coords[axis+1:]))
        direction = -1
    else:
        return point

    sols = solve(search.diff(t) == 0, t)   # integrate funcs later
    if not sols:
        return point

    try:
        sol = sols[0].rhs()
    except:
        sol = sols[0]
    delta = float(N(sol)) * direction

    ### chatgpt : try next time
    ### delta = float(N(sols[0].rhs() if hasattr(sols[0], 'rhs') else sols[0])) * direction
    
    # update
    coords[axis] += delta
    
    return tuple(coords)


In [121]:
def univariate_search(func, init=(0, 0), tol=1e-6, max_iter=100):
    
    current = init
    for _ in range(max_iter):
        # along e1
        next_x = axis_search(func, current, axis=0)
        # now aong e2
        next_pt = axis_search(func, next_x, axis=1)

        move_mag = ((next_pt[0] - current[0])**2 + (next_pt[1] - current[1])**2)**0.5
        if move_mag < tol:
            break
        current = next_pt
    return current


In [122]:
t0 = (0.0, 0.0)
opt = univariate_search(func, init=t0)
print(f"Computed minimum at: {opt}")

Computed minimum at: (1.3166290070706357, 2.4749435106059536)


# MultiDimensional Function Solver

In [123]:
f = lambda a,b,c: a**2 + b**2 + c**2 - a - 2*b - 3*c

In [13]:
def multi_axis_search(func, point, axis, step=1e-2):
    t = var('t')
    coords = list(point)
    base = float(func(*coords))

    coords[axis] = point[axis] + step
    forward = float(func(*coords))
    coords[axis] = point[axis] - step
    backward = float(func(*coords))
    coords[axis] = point[axis]

    if forward < backward and forward < base:
        search = func(*(coords[:axis] + [point[axis] + t] + coords[axis+1:]))
        direction =  1
    elif backward < forward and backward < base:
        search = func(*(coords[:axis] + [point[axis] - t] + coords[axis+1:]))
        direction = -1
    else:
        return point

    sols = solve(search.diff(t) == 0, t)
    if not sols:
        return point

    try:
        sol = sols[0].rhs()
    except:
        sol = sols[0]
    delta = float(N(sol)) * direction

    coords[axis] = point[axis] + delta
    return tuple(coords)


In [14]:
def MD_univariate_search(func, init, tol=1e-6, max_iter=100):

    point = tuple(init)
    n = len(point)
    for _ in range(max_iter):
        for i in range(n):
            point = multi_axis_search(func, point, axis=i)
            
        shift = sum((point[i] - init[i])**2 for i in range(n))**0.5
        if shift < tol:
            break
        init = point
    return point


In [126]:
t0 = (0.0, 0.0,0.0)
opt = MD_univariate_search(f, init=t0)
print(f"Computed minimum at: {opt}")

Computed minimum at: (0.5, 1.0, 1.5)


# Tests

### Sphere2D (0 0)

In [128]:
s = lambda x,y: x**2 + y**2

In [129]:
start = time.time()
res = MD_univariate_search(s, init=(0.5, -0.5), tol=1e-6, max_iter=200)
duration = time.time() - start
min = (0.00000, 0.00000) # known
error = np.linalg.norm(np.array(found) - np.array(true_min))

print(f"Found Minimum : {found}   ,  Error Norm : {error:.2e}   ,  Time (s) : {duration:.3f}")


Found Minimum : (0.0, 0.0)   ,  Error Norm : 1.41e+00   ,  Time (s) : 0.006


In [130]:
surface = plot3d(s, (x, -2, 2), (y, -2, 2) ,plot_points = 200, color='lightblue', opacity=0.8)
surface.show()

Graphics3d Object


In [131]:
min_dot = point3d((0, 0, 0), size=30, color='red', legend_label='Minimum')
min_dot.show()

Graphics3d Object


In [132]:
(surface + min_dot).show(frame=True, axes=True)

Graphics3d Object


In [142]:
# residuals = []
# point = (100)
# for i in range(15):
#     prev = point
#     point = axis_search(s, point, axis=i%2)
#     res = np.linalg.norm(np.array(point))
#     residuals.append(res)

In [143]:
# plt.figure()
# plt.plot(range(1, len(residuals)+1), residuals)
# plt.xlabel("Iteration")
# plt.ylabel("Residual (distance to origin)")
# plt.title("Convergence on Sphere 2D")
# plt.show()

### Rosenbrock Banana Function  (1 1)

In [11]:
banana = lambda a,b: 100*(b - a**2)**2 + (1 - a)**2

In [15]:
start = time.time()
res = MD_univariate_search(b, init=(0.5, -0.5), tol=1e-6, max_iter=200)
duration = time.time() - start
min = (0.00000, 0.00000) # known
error = np.linalg.norm(np.array(found) - np.array(true_min))

print(f"Found Minimum : {found}   ,  Error Norm : {error:.2e}   ,  Time (s) : {duration:.3f}")


TypeError: Substitution using function-call syntax and unnamed arguments has been removed. You can use named arguments instead, like EXPR(x=..., y=...)

### Rastringin  (0 0)

In [137]:
r = lambda a,b: 20 + a**2 - 10*np.cos(2*np.pi*a) + b**2 - 10*np.cos(2*np.pi*b)

In [138]:
start = time.time()
res = MD_univariate_search(b, init=(0.5, -0.5), tol=1e-6, max_iter=200)
duration = time.time() - start
min = (0.00000, 0.00000) # known
error = np.linalg.norm(np.array(found) - np.array(true_min))

print(f"Found Minimum : {found}   ,  Error Norm : {error:.2e}   ,  Time (s) : {duration:.3f}")


TypeError: unable to convert 0.504949534628357 - 0.710685228268544*I to float; use abs() or real_part() as desired

### Beale (3 2)

In [139]:
l = lambda a,b: (1.5 - a + a*b)**2 + (2.25 - a + a*b**2)**2 + (2.625 - a + a*b**3)**2

In [140]:
start = time.time()
res = MD_univariate_search(l, init=(0.5, -0.5), tol=1e-6, max_iter=200)
duration = time.time() - start
min = (0.00000, 0.00000) # known
error = np.linalg.norm(np.array(found) - np.array(true_min))

print(f"Found Minimum : {found}   ,  Error Norm : {error:.2e}   ,  Time (s) : {duration:.3f}")


TypeError: cannot evaluate symbolic expression numerically